In [1]:
import pandas as pd
import pymongo, tqdm
from collections import defaultdict

client = pymongo.MongoClient('mongodb://localhost:27017')

# OLD DB (5.0.3)

In [86]:
import tqdm
def sorted_pair(s, o):
   return "_".join(sorted([s, o]))

db = client['DrugBanks']
collection = db['DB_503']
#cursor = collection.find({'$or':[{"s":{"$in":wanted_drugs_all}}, 
#                                 {"o":{"$in":wanted_drugs_all}}]})

cursor = collection.find()

print("Duplicate Pairs")
n_old = 0
found_old = []
for doc in tqdm.tqdm_notebook(cursor):
    n_old += 1
    cur = sorted_pair(doc['s'], doc['o'])
    found_old.append(cur)

print "Αll pairs: %d" % n_old
found_old = list(set(found_old))
print "Unique pairs: %d" %len(found_old)

unique_dbs_old = set()
for k in set(found_old):
    unique_dbs_old.add(k.split("_")[0])
    unique_dbs_old.add(k.split("_")[1])
print "Unique DB ids: %d" %len(unique_dbs_old)

Duplicate Pairs



Αll pairs: 581055
Unique pairs: 296326
Unique DB ids: 2551


# NEW DB

In [87]:
db = client['DrugBanks']
collection = db['DB_531']
# cursor = collection.find({'$or':[{"s":{"$in":wanted_drugs_all}}, 
#                                  {"o":{"$in":wanted_drugs_all}}]})
cursor = collection.find()
print("Duplicate Pairs")
n = 0
found = []
for doc in tqdm.tqdm_notebook(cursor):
    n += 1
    cur = sorted_pair(doc['s'], doc['o'])
    found.append(cur)

print "Αll pairs: %d" % n
found = list(set(found))
print "Unique pairs: %d" %len(found)

unique_dbs = set()
for k in set(found):
    unique_dbs.add(k.split("_")[0])
    unique_dbs.add(k.split("_")[1])
print "Unique DB ids: %d" %len(unique_dbs)

Duplicate Pairs



Αll pairs: 2641889
Unique pairs: 1392809
Unique DB ids: 3814


In [88]:
intersection = set(found).intersection(found_old)
new_not_in_old = set(found).difference(found_old)
old_not_in_new = set(found_old).difference(found)
# SANITY CHECK. EITHER IN 5.0.3 or not
assert len(new_not_in_old) + len(intersection) == len(found)
# SANITY CHECK. EITHER IN 5.1.3 or not (THIS DON'T KNOW WHY, BUT SHOULD BE 0!)
assert len(new_not_in_old) + len(intersection) == len(found)

print("DB 5.1.3: %d ALL | %d (%0.2f %%) UNIQUE DDIS. ALSO IN DB 5.0.3: %d (%0.2f %%) | NEW: %d (%0.2f %%)" % (n, len(found), 100*len(found)/float(n), len(intersection), 100*len(intersection)/float(len(found)), len(new_not_in_old), 100*len(new_not_in_old)/float(len(found))))

print("DB 5.0.3: %d ALL | %d (%0.2f %%) UNIQUE DDIS. ALSO IN DB 5.1.3: %d (%0.2f %%) | NEW: %d (%0.2f %%)" % (n_old, len(found_old), 100*len(found_old)/float(n_old), len(intersection), 100*len(intersection)/float(len(found_old)), len(old_not_in_new), 100*len(old_not_in_new)/float(len(found_old))))


DB 5.1.3: 2641889 ALL | 1392809 (52.72 %) UNIQUE DDIS. ALSO IN DB 5.0.3: 236174 (16.96 %) | NEW: 1156635 (83.04 %)
DB 5.0.3: 581055 ALL | 296326 (51.00 %) UNIQUE DDIS. ALSO IN DB 5.1.3: 236174 (79.70 %) | NEW: 60152 (20.30 %)


# SAVE CSVs

### First save:
- as train the unique old *296326*
- test all the unique new ones not in old *1156635*

In [93]:
new_.shape

(1156635, 3)

In [89]:
old_ = [pair.split('_') for pair in sorted(found_old)]
old_ = pd.DataFrame(old_, columns=['h','t'])
old_['r'] = 4
old_ = old_[['h', 'r', 't']]
old_.to_csv("./train_OLD_STR_ALL.txt", header=False, index=False)
assert old_.shape[0] == len(found_old)

In [90]:
new_ = [pair.split('_') for pair in sorted(new_not_in_old)]
new_ = pd.DataFrame(new_, columns=['h','t'])
new_['r'] = 4
new_ = new_[['h', 'r', 't']]
new_.to_csv("./test_CLEAN_STR_ALL.txt", header=False, index=False)
assert new_.shape[0] == len(new_not_in_old)

# Filter using the Drugs that we have description for


In [46]:
import pandas as pd
df = pd.read_csv('../DB_BioBERT_TorchKGE/DB_Bert_Emb.csv', encoding='utf8')
db_ids = sorted(df['DB_ID'].unique().tolist())
db_name_2_id = dict(zip(db_ids,range(len(db_ids))))
print("%d Unique Drugs!" % len(db_ids))

12112 Unique Drugs!


In [65]:
def check_in_biobert(row, db_ids=db_ids):
    if row['h'] in db_ids and row['t'] in db_ids:
        return 1
    else:
        return 0
    
old_['BIOBERT'] = old_.apply(check_in_biobert, axis=1)
print("OLD WITH BIOBERT DETAILS:")
print(old_['BIOBERT'].value_counts())
new_['BIOBERT'] = new_.apply(check_in_biobert, axis=1)
print("NEW WITH BIOBERT DETAILS:")
print(new_['BIOBERT'].value_counts())


/home/kostas/.local/lib/python2.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


OLD WITH BIOBERT DETAILS:
1    290722
0      5588
Name: BIOBERT, dtype: int64
NEW WITH BIOBERT DETAILS:
1    1004167
0     152468
Name: BIOBERT, dtype: int64


### WILL SAVE THOSE WITH BIOBERT EMBEDDINGS (BOTH DRUGS!)
- as train the unique old *290722* 
- test all the unique new ones not in old *1004167*

In [66]:
old_ = old_[old_["BIOBERT"]==1]
new_ = new_[new_["BIOBERT"]==1]
del old_['BIOBERT']
del new_['BIOBERT']
print("WILL SAVE %d UNIQUE OLD ONES FOR TRAIN" % (old_.shape[0]))
print("WILL SAVE %d UNIQUE NEW ONES FOR TEST"% (new_.shape[0]))
old_.to_csv("./train_OLD_STR_BIOBERT.txt", header=False, index=False)
new_.to_csv("./test_CLEAN_STR_BIOBERT.txt", header=False, index=False)

WILL SAVE 290722 UNIQUE OLD ONES FOR TRAIN
WILL SAVE 1004167 UNIQUE NEW ONES FOR TEST


# Save the BIOBERT ONES USING INTEGER MAPPING

In [73]:
import numpy as np
# First get all pairs (train+test) so as to find which db id's are actually used!
db_ids_with_biobert = np.concatenate([old_['h'].unique(),old_['t'].unique(),new_['h'].unique(),new_['t'].unique()])
df_new = df[df["DB_ID"].isin(db_ids_with_biobert)].reset_index(drop=True)
db_name_2_id_new = dict(zip(df_new.DB_ID.values,range(len(df_new.DB_ID.values))))
df_new["DB_ID"] = df_new.index.values


In [81]:
#old_['h'] = old_['h'].map(db_name_2_id_new)
#old_['t'] = old_['t'].map(db_name_2_id_new)
new_['h'] = new_['h'].map(db_name_2_id_new)
new_['t'] = new_['t'].map(db_name_2_id_new)

In [84]:
old_.to_csv("./train_OLD_INT_BIOBERT.txt", header=False, index=False)
new_.to_csv("./test_CLEAN_INT_BIOBERT.txt", header=False, index=False)
df_new.to_csv("./DB_Bert_Emb_INT.csv", index=False, header=True)

In [107]:
#id_2_db_new = dict(zip(db_name_2_id_new.values(), db_name_2_id_new.keys()))
#df_new["DB_ID"] = df_new['DB_ID'].map(id_2_db_new)
df_new.to_csv("./DB_Bert_Emb_STR.csv", index=False, header=True)

In [101]:
pd.DataFrame({'DB_ID':db_name_2_id_new.keys(), "INDEX":db_name_2_id_new.values()}).sort_values('DB_ID').to_csv("./DB_Bert_Emb_INT__MAPPING.csv", index=False)

In [106]:
df_new

,0,1,2,3,4,5,6,7,8,9,...,759,760,761,762,763,764,765,766,767,DB_ID
0,0.202025,-0.006617,-0.099413,0.048500,0.060521,-0.073021,-0.090681,-0.000270,0.015176,-0.072919,...,-0.022322,-0.139573,-0.087268,0.079795,0.023646,-0.100570,0.163792,-0.016087,-0.022908,DB00001
1,0.177672,0.026007,-0.048232,0.018856,0.078177,0.000059,-0.337690,0.217661,-0.094409,-0.067416,...,0.053545,-0.057978,0.253709,-0.220694,-0.094370,0.025264,0.129294,-0.081248,-0.011695,DB00002
2,0.242237,0.051406,-0.031407,0.186637,0.066632,0.044037,-0.095817,-0.014136,0.198108,-0.016718,...,0.045745,0.036296,-0.138668,0.196352,-0.007737,-0.258840,0.366224,0.009603,0.081339,DB00004
3,0.226416,0.026122,-0.053552,0.251231,0.134691,0.050495,-0.148269,0.002328,-0.022043,-0.109596,...,-0.021331,0.066047,-0.195496,0.074613,0.002084,0.123117,0.145152,0.028520,0.002500,DB00005
4,0.257275,-0.033044,-0.031118,0.195232,0.170565,-0.095780,-0.081382,0.040114,0.052327,-0.131707,...,0.004533,-0.114376,-0.047823,0.143142,-0.042204,-0.000256,0.079256,0.105483,-0.058513,DB00006
5,0.089872,-0.104614,-0.017005,0.118278,0.163183,-0.079545,-0.081156,0.042572,-0.011747,0.022816,...,-0.034925,-0.162058,0.018929,0.069053,-0.161079,0.162147,0.177649,0.125209,0.022171,DB00007
6,0.030359,0.049868,-0.055352,0.058295,0.021681,-0.107721,-0.170107,0.183097,0.018565,0.021129,...,0.044376,-0.112331,-0.051223,0.146321,-0.098125,0.067621,0.183657,-0.059942,-0.010987,DB00008
7,0.258122,-0.010933,0.012392,0.174634,0.166051,-0.054834,-0.148571,-0.029997,0.062659,0.001520,...,-0.005217,-0.124742,-0.097660,0.076410,0.037419,0.067458,0.303003,0.093877,0.086248,DB00009
8,0.262155,0.004658,-0.080868,0.239816,0.321597,-0.123344,0.008519,0.079017,0.012092,-0.053687,...,-0.028001,-0.064884,-0.145939,0.155519,-0.029919,-0.065530,0.327868,-0.126697,0.046933,DB00011
9,-0.008348,0.101089,-0.040840,0.216455,0.040123,0.119392,-0.093857,0.112522,0.035863,0.079338,...,-0.050844,-0.215099,-0.128459,0.103433,-0.158186,0.041478,0.243607,0.104221,-0.051739,DB00012
